In [4]:
import os
import re
import argparse
import pickle
import pandas as pd
import numpy as np
import scipy.sparse as ssp
import dgl
import torch
import torchtext
from builder import PandasGraphBuilder
from data_utils import *

ImportError: dlopen(/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so, 2): Symbol not found: __ZNK3c104Type14isSubtypeOfExtERKNSt3__110shared_ptrIS0_EEPNS1_13basic_ostreamIcNS1_11char_traitsIcEEEE
  Referenced from: /Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so
  Expected in: /Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/torch/lib/libtorch_cpu.dylib
 in /Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/torchtext/_torchtext.so